In [2]:
import ee

In [4]:
ee.Authenticate() # If credentials already exist, they are automatically reused
# ee.Authenticate(force=True) # to create new credentials
ee.Initialize()

In [5]:
import geemap
Map = geemap.Map()


In [6]:
import os
os.chdir(r'\\172.16.20.21\wrd_p_igbp\IGBP_PROJECT_2023-24\IGBP_Project_files\Data\ET\MODIS_500m')

# Monthly ET 

In [8]:
# Define the shapefile as a FeatureCollection
shapefile = ee.FeatureCollection('projects/ee-rachitknl/assets/Rectangle')
# Get the bounding box of the shapefile
bounding_box = shapefile.geometry().bounds()

# roi_split = bounding_box.split(10)

# Define the CHIRPS collection
collection = ee.ImageCollection('MODIS/061/MOD16A2GF').select('ET')

# Define the years range
start_year = 2022
end_year = 2024

# Create a folder to save the downloaded data
output_folder = 'Evapotranspiration'

# Loop through the years
for year in range(start_year, end_year + 1):
    # Create a folder for the current year
    year_folder = output_folder + str(year) + '/'
   
    # Loop through the months
    for month in range(1, 13):
        # Filter the collection by year and month
        filtered_collection = collection \
            .filter(ee.Filter.calendarRange(year, year, 'year')) \
            .filter(ee.Filter.calendarRange(month, month, 'month'))
       
        # Check if the filtered collection has any images
        if filtered_collection.size().getInfo() > 0:
            # Multiply each image by scaling factor of 0.1
            scaled_collection = filtered_collection.map(lambda image: image.multiply(0.1))
           
            # Calculate the composite by summing up the scaled images
            composite = scaled_collection.sum()
           
            # Resample the composite to 500m resolution
            resampled_composite = composite \
                .reproject(crs=composite.projection(), scale=500) \
                .toDouble()  # Convert to floating-point type
            
            projection = resampled_composite.select(0).projection().getInfo()
            crs_transform = projection['transform']
            # Define the output filename
            output_filename = 'ET_' +  str(month).zfill(2)+ '_' + str(year)
            # Export the resampled composite as GeoTIFF
            # out_dir = os.path.expanduser(r'\\172.16.20.21\wrd_p_igbp\IGBP_PROJECT_2023-24\IGBP_Project_files\Data\ET\MODIS_500m')
            # geemap.ee_export_image_collection(scaled_collection, out_dir=out_dir, region=bounding_box)
            # for i, chunk in enumerate(roi_split):
            #     geemap.ee_export_image(
            #         resampled_composite.clip(chunk),
            #         filename= 'chunk_{i}'+'ET_' +  str(month).zfill(2)+ '_' + str(year) + '.tif',
            #         # region=bounding_box,
            #         scale=500,
            #         crs='EPSG:4326',
            #         crs_transform=crs_transform,
            #     )
            task = ee.batch.Export.image.toDrive(
            image=resampled_composite,
            description='ET_' +  str(month).zfill(2)+ '_' + str(year),
            folder=output_folder,
            fileNamePrefix=output_filename,
            region=bounding_box,
            scale=500,
            crs='EPSG:4326'
        )
            task.start()
        else:
            print("No data available for {}-{}".format(year, str(month).zfill(2)))

print("Export tasks submitted.")

No data available for 2024-01
No data available for 2024-02
No data available for 2024-03
No data available for 2024-04
No data available for 2024-05
No data available for 2024-06
No data available for 2024-07
No data available for 2024-08
No data available for 2024-09
No data available for 2024-10
No data available for 2024-11
No data available for 2024-12
Export tasks submitted.


In [16]:
shapefile.features

In [24]:
type(shapefile)

ee.featurecollection.FeatureCollection